In [13]:
import numpy as np
import cv2
import os
import torch as t
import visdom
import time

from torch import nn,optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils import data

vis = visdom.Visdom(env = 'grad_FC_txt')


In [3]:
kernel_ = np.array([[1/16,1/8,1/16],[1/8,1/4,1/8],[1/16,1/8,1/16]])

p = np.zeros((28,3,3))
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[0,1] = 1; p[0] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[0,2] = 1; p[1] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[1,0] = 1; p[2] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[1,2] = 1; p[3] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[2,0] = 1; p[4] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[2,1] = 1; p[5] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[2,2] = 1; p[6] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[0,2] = 1; p[7] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[1,0] = 1; p[8] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[1,2] = 1; p[9] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[2,0] = 1; p[10] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[2,1] = 1; p[11] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[2,2] = 1; p[12] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[1,0] = 1; p[13] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[1,2] = 1; p[14] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[2,0] = 1; p[15] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[2,1] = 1; p[16] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[2,2] = 1; p[17] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[1,2] = 1; p[18] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[2,0] = 1; p[19] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[2,1] = 1; p[20] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[2,2] = 1; p[21] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,2] = 1;zeros[2,0] = 1; p[22] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,2] = 1;zeros[2,1] = 1; p[23] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,2] = 1;zeros[2,2] = 1; p[24] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[2,0] = 1;zeros[2,1] = 1; p[25] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[2,0] = 1;zeros[2,2] = 1; p[26] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[2,1] = 1;zeros[2,2] = 1; p[27] = zeros


In [4]:
#这个模块用来对所有的图像求的最大值和最小值用来归一化。
#这个模块对图片进行分块求均值和标准差，分为6*6
img_2grad = np.zeros((28,40,40,3))
var_array = np.zeros((84))
var_array1 = np.zeros((28))
aver_array = np.zeros((84))
aver_array1 = np.zeros((28))
var_array_max_all = 0
var_array_min_all = 100000
aver_array_max_all = 0
aver_array_min_all = 100000
var_array_max1_all = 0
var_array_min1_all = 100000
aver_array_max1_all = 0
aver_array_min1_all = 100000
dir_path = "D:\\newfolder43_1_x"
dirs = os.listdir(dir_path)
for i in range(len(dirs)):
    pic_names = os.listdir(dir_path+"\\"+dirs[i])
    for n in range(len(pic_names)):
        pic_path = dir_path+"\\"+dirs[i]+"\\"+pic_names[n]
        print(pic_path)
        #pic_path_gbk = pic_path.encode("gbk")
        #pic_path_ascii = pic_path_gbk.decode()
        #image = cv2.imread(pic_path_ascii)
        image=cv2.imdecode(np.fromfile(pic_path,dtype=np.uint8),-1)
        all_result_array = np.array([])
        all_var_array = np.array([])
        all_aver_array = np.array([])
        all_var_array1 = np.array([])
        all_aver_array1 = np.array([])
        for hor_num in range(3):
            for col_num in range(3):
                image1 = image[(40*hor_num):(40*(hor_num+1)),(40*col_num):(40*(col_num+1))]
                image_gaussion = cv2.filter2D(image1,cv2.CV_16S,kernel_)
                for m in range(28):
                    img_2grad[m] = cv2.filter2D(image_gaussion,cv2.CV_16S,p[m])
                    var_array[3*m] = img_2grad[m][:,:,0].std()
                    var_array[3*m+1] = img_2grad[m][:,:,1].std()
                    var_array[3*m+2] = img_2grad[m][:,:,2].std()
                    aver_array[3*m] = img_2grad[m][:,:,0].mean()
                    aver_array[3*m+1] = img_2grad[m][:,:,1].mean()
                    aver_array[3*m+2] = img_2grad[m][:,:,2].mean()
                    var_array1[m] = img_2grad[m].std()
                    aver_array1[m] = img_2grad[m].mean()
                
                all_var_array = np.hstack((all_var_array,var_array))
                all_aver_array = np.hstack((all_aver_array,aver_array))   
                all_var_array1 = np.hstack((all_var_array1,var_array1))
                all_aver_array1 = np.hstack((all_aver_array1,aver_array1))
                
        all_var_array_max = all_var_array.max()
        all_var_array_min = all_var_array.min()
        all_aver_array_max = all_aver_array.max()
        all_aver_array_min = all_aver_array.min()
        all_var_array_max1 = all_var_array1.max()
        all_var_array_min1 = all_var_array1.min()
        all_aver_array_max1 = all_aver_array1.max()
        all_aver_array_min1 = all_aver_array1.min()
        

        
        var_array_max_all = max(var_array_max_all,all_var_array_max)
        var_array_min_all = min(var_array_min_all,all_var_array_min)
        aver_array_max_all = max(aver_array_max_all,all_aver_array_max)
        aver_array_min_all = min(aver_array_min_all,all_aver_array_min)
        var_array_max1_all = max(var_array_max1_all,all_var_array_max1)
        var_array_min1_all = min(var_array_min1_all,all_var_array_min1)
        aver_array_max1_all = max(aver_array_max1_all,all_aver_array_max1)
        aver_array_min1_all = min(aver_array_min1_all,all_aver_array_min1)
print(var_array_max_all,var_array_min_all,aver_array_max_all,aver_array_min_all,var_array_max1_all,var_array_min1_all,aver_array_max1_all,aver_array_min1_all)        

D:\newfolder43_1_x\clear_face\picture100_147_.png
D:\newfolder43_1_x\clear_face\picture100_1595_.png
D:\newfolder43_1_x\clear_face\picture100_1599_.png
D:\newfolder43_1_x\clear_face\picture100_1601_.png
D:\newfolder43_1_x\clear_face\picture100_1614_.png
D:\newfolder43_1_x\clear_face\picture100_1623_.png
D:\newfolder43_1_x\clear_face\picture100_1626_.png
D:\newfolder43_1_x\clear_face\picture100_1648_.png
D:\newfolder43_1_x\clear_face\picture100_1710_.png
D:\newfolder43_1_x\clear_face\picture100_1712_.png
D:\newfolder43_1_x\clear_face\picture100_1714_.png
D:\newfolder43_1_x\clear_face\picture100_1720_.png
D:\newfolder43_1_x\clear_face\picture100_1723_.png
D:\newfolder43_1_x\clear_face\picture100_1726_.png
D:\newfolder43_1_x\clear_face\picture100_1741_.png
D:\newfolder43_1_x\clear_face\picture100_1745_.png
D:\newfolder43_1_x\clear_face\picture100_1751_.png
D:\newfolder43_1_x\clear_face\picture100_1809_.png
D:\newfolder43_1_x\clear_face\picture100_1813_.png
D:\newfolder43_1_x\clear_face\pi

D:\newfolder43_1_x\clear_face\picture100_4510_.png
D:\newfolder43_1_x\clear_face\picture100_4511_.png
D:\newfolder43_1_x\clear_face\picture100_4516_.png
D:\newfolder43_1_x\clear_face\picture100_451_.png
D:\newfolder43_1_x\clear_face\picture100_4526_.png
D:\newfolder43_1_x\clear_face\picture100_4530_.png
D:\newfolder43_1_x\clear_face\picture100_4534_.png
D:\newfolder43_1_x\clear_face\picture100_4538_.png
D:\newfolder43_1_x\clear_face\picture100_453_.png
D:\newfolder43_1_x\clear_face\picture100_4566_.png
D:\newfolder43_1_x\clear_face\picture100_4568_.png
D:\newfolder43_1_x\clear_face\picture100_4590_.png
D:\newfolder43_1_x\clear_face\picture100_4624_.png
D:\newfolder43_1_x\clear_face\picture100_4626_.png
D:\newfolder43_1_x\clear_face\picture100_4635_.png
D:\newfolder43_1_x\clear_face\picture100_4637_.png
D:\newfolder43_1_x\clear_face\picture100_463_.png
D:\newfolder43_1_x\clear_face\picture100_4641_.png
D:\newfolder43_1_x\clear_face\picture100_4643_.png
D:\newfolder43_1_x\clear_face\pict

D:\newfolder43_1_x\clear_face\picture100_5709_.png
D:\newfolder43_1_x\clear_face\picture100_5714_.png
D:\newfolder43_1_x\clear_face\picture100_5720_.png
D:\newfolder43_1_x\clear_face\picture100_5724_.png
D:\newfolder43_1_x\clear_face\picture100_5739_.png
D:\newfolder43_1_x\clear_face\picture100_5743_.png
D:\newfolder43_1_x\clear_face\picture100_5774_.png
D:\newfolder43_1_x\clear_face\picture100_5775_.png
D:\newfolder43_1_x\clear_face\picture100_5776_.png
D:\newfolder43_1_x\clear_face\picture100_583_.png
D:\newfolder43_1_x\clear_face\picture100_5848_.png
D:\newfolder43_1_x\clear_face\picture100_5849_.png
D:\newfolder43_1_x\clear_face\picture100_584_.png
D:\newfolder43_1_x\clear_face\picture100_5850_.png
D:\newfolder43_1_x\clear_face\picture100_5851_.png
D:\newfolder43_1_x\clear_face\picture100_5852_.png
D:\newfolder43_1_x\clear_face\picture100_585_.png
D:\newfolder43_1_x\clear_face\picture100_586_.png
D:\newfolder43_1_x\clear_face\picture100_587_.png
D:\newfolder43_1_x\clear_face\pictur

D:\newfolder43_1_x\clear_face\picture1_11298_.png
D:\newfolder43_1_x\clear_face\picture1_11301_.png
D:\newfolder43_1_x\clear_face\picture1_11302_.png
D:\newfolder43_1_x\clear_face\picture1_11305_.png
D:\newfolder43_1_x\clear_face\picture1_11313_.png
D:\newfolder43_1_x\clear_face\picture1_11315_.png
D:\newfolder43_1_x\clear_face\picture1_11317_.png
D:\newfolder43_1_x\clear_face\picture1_11321_.png
D:\newfolder43_1_x\clear_face\picture1_11322_.png
D:\newfolder43_1_x\clear_face\picture1_11323_.png
D:\newfolder43_1_x\clear_face\picture1_11325_.png
D:\newfolder43_1_x\clear_face\picture1_11329_.png
D:\newfolder43_1_x\clear_face\picture1_11333_.png
D:\newfolder43_1_x\clear_face\picture1_11336_.png
D:\newfolder43_1_x\clear_face\picture1_11338_.png
D:\newfolder43_1_x\clear_face\picture1_11340_.png
D:\newfolder43_1_x\clear_face\picture1_11343_.png
D:\newfolder43_1_x\clear_face\picture1_11346_.png
D:\newfolder43_1_x\clear_face\picture1_11349_.png
D:\newfolder43_1_x\clear_face\picture1_1134_.png
D

D:\newfolder43_1_x\clear_face\picture1_13404_.png
D:\newfolder43_1_x\clear_face\picture1_13406_.png
D:\newfolder43_1_x\clear_face\picture1_13409_.png
D:\newfolder43_1_x\clear_face\picture1_13411_.png
D:\newfolder43_1_x\clear_face\picture1_13413_.png
D:\newfolder43_1_x\clear_face\picture1_13417_.png
D:\newfolder43_1_x\clear_face\picture1_13467_.png
D:\newfolder43_1_x\clear_face\picture1_13468_.png
D:\newfolder43_1_x\clear_face\picture1_13469_.png
D:\newfolder43_1_x\clear_face\picture1_13470_.png
D:\newfolder43_1_x\clear_face\picture1_13471_.png
D:\newfolder43_1_x\clear_face\picture1_13472_.png
D:\newfolder43_1_x\clear_face\picture1_13474_.png
D:\newfolder43_1_x\clear_face\picture1_13476_.png
D:\newfolder43_1_x\clear_face\picture1_13478_.png
D:\newfolder43_1_x\clear_face\picture1_13480_.png
D:\newfolder43_1_x\clear_face\picture1_13482_.png
D:\newfolder43_1_x\clear_face\picture1_13483_.png
D:\newfolder43_1_x\clear_face\picture1_13485_.png
D:\newfolder43_1_x\clear_face\picture1_13486_.png


...

D:\newfolder43_1_x\fuzzy_face1\picture_2327_.png
D:\newfolder43_1_x\fuzzy_face1\picture_232_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2339_.png
D:\newfolder43_1_x\fuzzy_face1\picture_233_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2344_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2349_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2354_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2355_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2356_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2357_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2358_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2359_.png
D:\newfolder43_1_x\fuzzy_face1\picture_235_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2360_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2361_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2362_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2363_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2364_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2365_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2366_.png
D:\newfolder43_1_x\fuzz

D:\newfolder43_1_x\fuzzy_face1\picture_2566_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2567_.png
D:\newfolder43_1_x\fuzzy_face1\picture_256_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2573_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2575_.png
D:\newfolder43_1_x\fuzzy_face1\picture_257_.png
D:\newfolder43_1_x\fuzzy_face1\picture_258_.png
D:\newfolder43_1_x\fuzzy_face1\picture_259_.png
D:\newfolder43_1_x\fuzzy_face1\picture_25_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2605_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2608_.png
D:\newfolder43_1_x\fuzzy_face1\picture_260_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2618_.png
D:\newfolder43_1_x\fuzzy_face1\picture_261_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2623_.png
D:\newfolder43_1_x\fuzzy_face1\picture_262_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2630_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2631_.png
D:\newfolder43_1_x\fuzzy_face1\picture_2634_.png
D:\newfolder43_1_x\fuzzy_face1\picture_263_.png
D:\newfolder43_1_x\fuzzy_face1

D:\newfolder43_1_x\fuzzy_face1\picture_317_.png
D:\newfolder43_1_x\fuzzy_face1\picture_318_.png
D:\newfolder43_1_x\fuzzy_face1\picture_31_.png
D:\newfolder43_1_x\fuzzy_face1\picture_3202_.png
D:\newfolder43_1_x\fuzzy_face1\picture_3203_.png
D:\newfolder43_1_x\fuzzy_face1\picture_3204_.png
D:\newfolder43_1_x\fuzzy_face1\picture_3205_.png
D:\newfolder43_1_x\fuzzy_face1\picture_3207_.png
D:\newfolder43_1_x\fuzzy_face1\picture_320_.png
D:\newfolder43_1_x\fuzzy_face1\picture_3210_.png
D:\newfolder43_1_x\fuzzy_face1\picture_3213_.png
D:\newfolder43_1_x\fuzzy_face1\picture_3216_.png
D:\newfolder43_1_x\fuzzy_face1\picture_3217_.png
D:\newfolder43_1_x\fuzzy_face1\picture_321_.png
D:\newfolder43_1_x\fuzzy_face1\picture_3221_.png
D:\newfolder43_1_x\fuzzy_face1\picture_3224_.png
D:\newfolder43_1_x\fuzzy_face1\picture_322_.png
D:\newfolder43_1_x\fuzzy_face1\picture_3237_.png
D:\newfolder43_1_x\fuzzy_face1\picture_323_.png
D:\newfolder43_1_x\fuzzy_face1\picture_3240_.png
D:\newfolder43_1_x\fuzzy_fac

D:\newfolder43_1_x\fuzzy_face1\picture_495_.png
D:\newfolder43_1_x\fuzzy_face1\picture_497_.png
D:\newfolder43_1_x\fuzzy_face1\picture_498_.png
D:\newfolder43_1_x\fuzzy_face1\picture_49_.png
D:\newfolder43_1_x\fuzzy_face1\picture_502_.png
D:\newfolder43_1_x\fuzzy_face1\picture_503_.png
D:\newfolder43_1_x\fuzzy_face1\picture_504_.png
D:\newfolder43_1_x\fuzzy_face1\picture_505_.png
D:\newfolder43_1_x\fuzzy_face1\picture_506_.png
D:\newfolder43_1_x\fuzzy_face1\picture_50_.png
D:\newfolder43_1_x\fuzzy_face1\picture_515_.png
D:\newfolder43_1_x\fuzzy_face1\picture_51_.png
D:\newfolder43_1_x\fuzzy_face1\picture_520_.png
D:\newfolder43_1_x\fuzzy_face1\picture_521_.png
D:\newfolder43_1_x\fuzzy_face1\picture_523_.png
D:\newfolder43_1_x\fuzzy_face1\picture_524_.png
D:\newfolder43_1_x\fuzzy_face1\picture_525_.png
D:\newfolder43_1_x\fuzzy_face1\picture_526_.png
D:\newfolder43_1_x\fuzzy_face1\picture_527_.png
D:\newfolder43_1_x\fuzzy_face1\picture_529_.png
D:\newfolder43_1_x\fuzzy_face1\picture_52_.

D:\newfolder43_1_x\fuzzy_face1\picture_759_.png
D:\newfolder43_1_x\fuzzy_face1\picture_75_.png
D:\newfolder43_1_x\fuzzy_face1\picture_760_.png
D:\newfolder43_1_x\fuzzy_face1\picture_761_.png
D:\newfolder43_1_x\fuzzy_face1\picture_762_.png
D:\newfolder43_1_x\fuzzy_face1\picture_763_.png
D:\newfolder43_1_x\fuzzy_face1\picture_764_.png
D:\newfolder43_1_x\fuzzy_face1\picture_765_.png
D:\newfolder43_1_x\fuzzy_face1\picture_766_.png
D:\newfolder43_1_x\fuzzy_face1\picture_767_.png
D:\newfolder43_1_x\fuzzy_face1\picture_768_.png
D:\newfolder43_1_x\fuzzy_face1\picture_769_.png
D:\newfolder43_1_x\fuzzy_face1\picture_770_.png
D:\newfolder43_1_x\fuzzy_face1\picture_771_.png
D:\newfolder43_1_x\fuzzy_face1\picture_772_.png
D:\newfolder43_1_x\fuzzy_face1\picture_773_.png
D:\newfolder43_1_x\fuzzy_face1\picture_774_.png
D:\newfolder43_1_x\fuzzy_face1\picture_775_.png
D:\newfolder43_1_x\fuzzy_face1\picture_776_.png
D:\newfolder43_1_x\fuzzy_face1\picture_777_.png
D:\newfolder43_1_x\fuzzy_face1\picture_77

In [41]:
var_array_max_all = 88.76562225180422
var_array_min_all = 0.0
aver_array_max_all = 27.6975
aver_array_min_all = -28.7825
var_array_max1_all = 88.41747752926956
var_array_min1_all = 0.0
aver_array_max1_all = 26.2775
aver_array_min1_all = -27.200833333333332

In [5]:
#带0.5的normalize
img_2grad = np.zeros((28,40,40,3))
var_array = np.zeros((84))
var_array1 = np.zeros((28))
aver_array = np.zeros((84))
aver_array1 = np.zeros((28))
var_array_max_all = 0
var_array_min_all = 100000
aver_array_max_all = 0
aver_array_min_all = 100000
var_array_max1_all = 0
var_array_min1_all = 100000
aver_array_max1_all = 0
aver_array_min1_all = 100000
dir_path = "D:\\newfolder43_1"
save_path = "C:\\newfolder43_1_txt6"
dirs = os.listdir(dir_path)
for i in range(len(dirs)):
    pic_names = os.listdir(dir_path+"\\"+dirs[i])
    for n in range(len(pic_names)):
        pic_path = dir_path+"\\"+dirs[i]+"\\"+pic_names[n]
        print(pic_path)
        #pic_path_gbk = pic_path.encode("gbk")
        #pic_path_ascii = pic_path_gbk.decode()
        #image = cv2.imread(pic_path_ascii)
        image=cv2.imdecode(np.fromfile(pic_path,dtype=np.uint8),-1)
        all_result_array = np.array([])
        all_var_array = np.array([])
        all_aver_array = np.array([])
        all_var_array1 = np.array([])
        all_aver_array1 = np.array([])
        for hor_num in range(3):
            for col_num in range(3):
                image1 = image[(40*hor_num):(40*(hor_num+1)),(40*col_num):(40*(col_num+1))]
                image_gaussion = cv2.filter2D(image1,cv2.CV_16S,kernel_)
                for m in range(28):
                    img_2grad[m] = cv2.filter2D(image_gaussion,cv2.CV_16S,p[m])
                    var_array[3*m] = img_2grad[m][:,:,0].std()
                    var_array[3*m+1] = img_2grad[m][:,:,1].std()
                    var_array[3*m+2] = img_2grad[m][:,:,2].std()
                    #aver_array[3*m] = img_2grad[m][:,:,0].mean()
                    #aver_array[3*m+1] = img_2grad[m][:,:,1].mean()
                    #aver_array[3*m+2] = img_2grad[m][:,:,2].mean()
                    var_array1[m] = img_2grad[m].std()
                    #aver_array1[m] = img_2grad[m].mean()
                
                all_var_array = np.hstack((all_var_array,var_array))
                #all_aver_array = np.hstack((all_aver_array,aver_array))   
                all_var_array1 = np.hstack((all_var_array1,var_array1))
                #all_aver_array1 = np.hstack((all_aver_array1,aver_array1))
                        
        all_var_array = np.true_divide((all_var_array-var_array_min_all),(var_array_max_all-var_array_min_all))
        all_aver_array = np.true_divide((all_aver_array-aver_array_min_all),(aver_array_max_all-aver_array_min_all))
        all_var_array1 = np.true_divide((all_var_array1-var_array_min1_all),(var_array_max1_all-var_array_min1_all))
        all_aver_array1 = np.true_divide((all_aver_array1-aver_array_min1_all),(aver_array_max1_all-aver_array_min1_all))
        
        #all_var_array = np.true_divide((all_var_array-0.5),0.5) 
        #all_aver_array = np.true_divide((all_aver_array-0.5),0.5)
        #all_var_array1 = np.true_divide((all_var_array1-0.5),0.5) 
        #all_aver_array1 = np.true_divide((all_aver_array1-0.5),0.5)
        result_array = np.hstack((all_var_array,all_aver_array,all_var_array1,all_aver_array1))
        name = save_path+"\\"+dirs[i]+"\\"+(pic_names[n].split("."))[0]+".txt"
        fw = open(name,"w")
        fw.write(','.join(str(num) for num in result_array))
        fw.close()

D:\newfolder43_1\clear_face\picture100_147_.png
D:\newfolder43_1\clear_face\picture100_1595_.png
D:\newfolder43_1\clear_face\picture100_1599_.png
D:\newfolder43_1\clear_face\picture100_1601_.png
D:\newfolder43_1\clear_face\picture100_1614_.png
D:\newfolder43_1\clear_face\picture100_1623_.png
D:\newfolder43_1\clear_face\picture100_1626_.png
D:\newfolder43_1\clear_face\picture100_1648_.png
D:\newfolder43_1\clear_face\picture100_1710_.png
D:\newfolder43_1\clear_face\picture100_1712_.png
D:\newfolder43_1\clear_face\picture100_1714_.png
D:\newfolder43_1\clear_face\picture100_1720_.png
D:\newfolder43_1\clear_face\picture100_1723_.png
D:\newfolder43_1\clear_face\picture100_1726_.png
D:\newfolder43_1\clear_face\picture100_1741_.png
D:\newfolder43_1\clear_face\picture100_1745_.png
D:\newfolder43_1\clear_face\picture100_1751_.png
D:\newfolder43_1\clear_face\picture100_1809_.png
D:\newfolder43_1\clear_face\picture100_1813_.png
D:\newfolder43_1\clear_face\picture100_1815_.png
D:\newfolder43_1\clea

D:\newfolder43_1\clear_face\picture100_4534_.png
D:\newfolder43_1\clear_face\picture100_4538_.png
D:\newfolder43_1\clear_face\picture100_453_.png
D:\newfolder43_1\clear_face\picture100_4566_.png
D:\newfolder43_1\clear_face\picture100_4568_.png
D:\newfolder43_1\clear_face\picture100_4590_.png
D:\newfolder43_1\clear_face\picture100_4624_.png
D:\newfolder43_1\clear_face\picture100_4626_.png
D:\newfolder43_1\clear_face\picture100_4635_.png
D:\newfolder43_1\clear_face\picture100_4637_.png
D:\newfolder43_1\clear_face\picture100_463_.png
D:\newfolder43_1\clear_face\picture100_4641_.png
D:\newfolder43_1\clear_face\picture100_4643_.png
D:\newfolder43_1\clear_face\picture100_4645_.png
D:\newfolder43_1\clear_face\picture100_4646_.png
D:\newfolder43_1\clear_face\picture100_4670_.png
D:\newfolder43_1\clear_face\picture100_4672_.png
D:\newfolder43_1\clear_face\picture100_4677_.png
D:\newfolder43_1\clear_face\picture100_4679_.png
D:\newfolder43_1\clear_face\picture100_4681_.png
D:\newfolder43_1\clear

D:\newfolder43_1\clear_face\picture100_584_.png
D:\newfolder43_1\clear_face\picture100_5850_.png
D:\newfolder43_1\clear_face\picture100_5851_.png
D:\newfolder43_1\clear_face\picture100_5852_.png
D:\newfolder43_1\clear_face\picture100_585_.png
D:\newfolder43_1\clear_face\picture100_586_.png
D:\newfolder43_1\clear_face\picture100_587_.png
D:\newfolder43_1\clear_face\picture100_6009_.png
D:\newfolder43_1\clear_face\picture100_6011_.png
D:\newfolder43_1\clear_face\picture100_6013_.png
D:\newfolder43_1\clear_face\picture100_6014_.png
D:\newfolder43_1\clear_face\picture100_6015_.png
D:\newfolder43_1\clear_face\picture100_6016_.png
D:\newfolder43_1\clear_face\picture100_6017_.png
D:\newfolder43_1\clear_face\picture100_6018_.png
D:\newfolder43_1\clear_face\picture100_6030_.png
D:\newfolder43_1\clear_face\picture100_6031_.png
D:\newfolder43_1\clear_face\picture100_6032_.png
D:\newfolder43_1\clear_face\picture100_6033_.png
D:\newfolder43_1\clear_face\picture100_6034_.png
D:\newfolder43_1\clear_f

D:\newfolder43_1\clear_face\picture1_11358_.png
D:\newfolder43_1\clear_face\picture1_11360_.png
D:\newfolder43_1\clear_face\picture1_11363_.png
D:\newfolder43_1\clear_face\picture1_11365_.png
D:\newfolder43_1\clear_face\picture1_11368_.png
D:\newfolder43_1\clear_face\picture1_11370_.png
D:\newfolder43_1\clear_face\picture1_11372_.png
D:\newfolder43_1\clear_face\picture1_11374_.png
D:\newfolder43_1\clear_face\picture1_11376_.png
D:\newfolder43_1\clear_face\picture1_11378_.png
D:\newfolder43_1\clear_face\picture1_1137_.png
D:\newfolder43_1\clear_face\picture1_11380_.png
D:\newfolder43_1\clear_face\picture1_11382_.png
D:\newfolder43_1\clear_face\picture1_11391_.png
D:\newfolder43_1\clear_face\picture1_1139_.png
D:\newfolder43_1\clear_face\picture1_11421_.png
D:\newfolder43_1\clear_face\picture1_11422_.png
D:\newfolder43_1\clear_face\picture1_11424_.png
D:\newfolder43_1\clear_face\picture1_11427_.png
D:\newfolder43_1\clear_face\picture1_11428_.png
D:\newfolder43_1\clear_face\picture1_1142_

D:\newfolder43_1\clear_face\picture1_1400_.png
D:\newfolder43_1\clear_face\picture1_1403_.png
D:\newfolder43_1\clear_face\picture1_1407_.png
D:\newfolder43_1\clear_face\picture1_14086_.png
D:\newfolder43_1\clear_face\picture1_1409_.png
D:\newfolder43_1\clear_face\picture1_1420_.png
D:\newfolder43_1\clear_face\picture1_1421_.png
D:\newfolder43_1\clear_face\picture1_1424_.png
D:\newfolder43_1\clear_face\picture1_14280_.png
D:\newfolder43_1\clear_face\picture1_14281_.png
D:\newfolder43_1\clear_face\picture1_14282_.png
D:\newfolder43_1\clear_face\picture1_14283_.png
D:\newfolder43_1\clear_face\picture1_1429_.png
D:\newfolder43_1\clear_face\picture1_1432_.png
D:\newfolder43_1\clear_face\picture1_14713_.png
D:\newfolder43_1\clear_face\picture1_14716_.png
D:\newfolder43_1\clear_face\picture1_14718_.png
D:\newfolder43_1\clear_face\picture1_14721_.png
D:\newfolder43_1\clear_face\picture1_14725_.png
D:\newfolder43_1\clear_face\picture1_14727_.png
D:\newfolder43_1\clear_face\picture1_14729_.png
D

...

D:\newfolder43_1\fuzzy_face\picture_5726_.png
D:\newfolder43_1\fuzzy_face\picture_5733_.png
D:\newfolder43_1\fuzzy_face\picture_5735_.png
D:\newfolder43_1\fuzzy_face\picture_573_.png
D:\newfolder43_1\fuzzy_face\picture_5740_.png
D:\newfolder43_1\fuzzy_face\picture_5741_.png
D:\newfolder43_1\fuzzy_face\picture_5746_.png
D:\newfolder43_1\fuzzy_face\picture_5747_.png
D:\newfolder43_1\fuzzy_face\picture_5751_.png
D:\newfolder43_1\fuzzy_face\picture_5752_.png
D:\newfolder43_1\fuzzy_face\picture_5756_.png
D:\newfolder43_1\fuzzy_face\picture_5757_.png
D:\newfolder43_1\fuzzy_face\picture_5763_.png
D:\newfolder43_1\fuzzy_face\picture_5765_.png
D:\newfolder43_1\fuzzy_face\picture_5768_.png
D:\newfolder43_1\fuzzy_face\picture_576_.png
D:\newfolder43_1\fuzzy_face\picture_5770_.png
D:\newfolder43_1\fuzzy_face\picture_5775_.png
D:\newfolder43_1\fuzzy_face\picture_5777_.png
D:\newfolder43_1\fuzzy_face\picture_5781_.png
D:\newfolder43_1\fuzzy_face\picture_5783_.png
D:\newfolder43_1\fuzzy_face\picture_

D:\newfolder43_1\fuzzy_face\picture_6824_.png
D:\newfolder43_1\fuzzy_face\picture_6942_.png
D:\newfolder43_1\fuzzy_face\picture_6944_.png
D:\newfolder43_1\fuzzy_face\picture_6945_.png
D:\newfolder43_1\fuzzy_face\picture_6948_.png
D:\newfolder43_1\fuzzy_face\picture_6950_.png
D:\newfolder43_1\fuzzy_face\picture_6952_.png
D:\newfolder43_1\fuzzy_face\picture_6975_.png
D:\newfolder43_1\fuzzy_face\picture_6980_.png
D:\newfolder43_1\fuzzy_face\picture_6982_.png
D:\newfolder43_1\fuzzy_face\picture_6990_.png
D:\newfolder43_1\fuzzy_face\picture_6991_.png
D:\newfolder43_1\fuzzy_face\picture_6992_.png
D:\newfolder43_1\fuzzy_face\picture_6993_.png
D:\newfolder43_1\fuzzy_face\picture_6994_.png
D:\newfolder43_1\fuzzy_face\picture_6995_.png
D:\newfolder43_1\fuzzy_face\picture_6997_.png
D:\newfolder43_1\fuzzy_face\picture_6999_.png
D:\newfolder43_1\fuzzy_face\picture_6_.png
D:\newfolder43_1\fuzzy_face\picture_7000_.png
D:\newfolder43_1\fuzzy_face\picture_7002_.png
D:\newfolder43_1\fuzzy_face\picture_7

D:\newfolder43_1\fuzzy_face\picture_7370_.png
D:\newfolder43_1\fuzzy_face\picture_7371_.png
D:\newfolder43_1\fuzzy_face\picture_7374_.png
D:\newfolder43_1\fuzzy_face\picture_7376_.png
D:\newfolder43_1\fuzzy_face\picture_7377_.png
D:\newfolder43_1\fuzzy_face\picture_7381_.png
D:\newfolder43_1\fuzzy_face\picture_7382_.png
D:\newfolder43_1\fuzzy_face\picture_7383_.png
D:\newfolder43_1\fuzzy_face\picture_7385_.png
D:\newfolder43_1\fuzzy_face\picture_7387_.png
D:\newfolder43_1\fuzzy_face\picture_7388_.png
D:\newfolder43_1\fuzzy_face\picture_7394_.png
D:\newfolder43_1\fuzzy_face\picture_7398_.png
D:\newfolder43_1\fuzzy_face\picture_7399_.png
D:\newfolder43_1\fuzzy_face\picture_7403_.png
D:\newfolder43_1\fuzzy_face\picture_7404_.png
D:\newfolder43_1\fuzzy_face\picture_7407_.png
D:\newfolder43_1\fuzzy_face\picture_740_.png
D:\newfolder43_1\fuzzy_face\picture_7411_.png
D:\newfolder43_1\fuzzy_face\picture_7412_.png
D:\newfolder43_1\fuzzy_face\picture_7413_.png
D:\newfolder43_1\fuzzy_face\picture

D:\newfolder43_1\fuzzy_face\picture_8131_.png
D:\newfolder43_1\fuzzy_face\picture_8132_.png
D:\newfolder43_1\fuzzy_face\picture_8183_.png
D:\newfolder43_1\fuzzy_face\picture_8185_.png
D:\newfolder43_1\fuzzy_face\picture_8187_.png
D:\newfolder43_1\fuzzy_face\picture_8189_.png
D:\newfolder43_1\fuzzy_face\picture_8197_.png
D:\newfolder43_1\fuzzy_face\picture_8203_.png
D:\newfolder43_1\fuzzy_face\picture_8206_.png
D:\newfolder43_1\fuzzy_face\picture_8209_.png
D:\newfolder43_1\fuzzy_face\picture_8212_.png
D:\newfolder43_1\fuzzy_face\picture_8215_.png
D:\newfolder43_1\fuzzy_face\picture_8216_.png
D:\newfolder43_1\fuzzy_face\picture_8222_.png
D:\newfolder43_1\fuzzy_face\picture_8225_.png
D:\newfolder43_1\fuzzy_face\picture_8226_.png
D:\newfolder43_1\fuzzy_face\picture_8241_.png
D:\newfolder43_1\fuzzy_face\picture_8268_.png
D:\newfolder43_1\fuzzy_face\picture_8270_.png
D:\newfolder43_1\fuzzy_face\picture_8294_.png
D:\newfolder43_1\fuzzy_face\picture_84_.png
D:\newfolder43_1\fuzzy_face\picture_

D:\newfolder43_1\fuzzy_face\picture_958_.png
D:\newfolder43_1\fuzzy_face\picture_9597_.png
D:\newfolder43_1\fuzzy_face\picture_9598_.png
D:\newfolder43_1\fuzzy_face\picture_959_.png
D:\newfolder43_1\fuzzy_face\picture_95_.png
D:\newfolder43_1\fuzzy_face\picture_9600_.png
D:\newfolder43_1\fuzzy_face\picture_9601_.png
D:\newfolder43_1\fuzzy_face\picture_9603_.png
D:\newfolder43_1\fuzzy_face\picture_9604_.png
D:\newfolder43_1\fuzzy_face\picture_9605_.png
D:\newfolder43_1\fuzzy_face\picture_9606_.png
D:\newfolder43_1\fuzzy_face\picture_9607_.png
D:\newfolder43_1\fuzzy_face\picture_9608_.png
D:\newfolder43_1\fuzzy_face\picture_9609_.png
D:\newfolder43_1\fuzzy_face\picture_9610_.png
D:\newfolder43_1\fuzzy_face\picture_9611_.png
D:\newfolder43_1\fuzzy_face\picture_9612_.png
D:\newfolder43_1\fuzzy_face\picture_9613_.png
D:\newfolder43_1\fuzzy_face\picture_9614_.png
D:\newfolder43_1\fuzzy_face\picture_9615_.png
D:\newfolder43_1\fuzzy_face\picture_9616_.png
D:\newfolder43_1\fuzzy_face\picture_96

In [1]:
a = 1
b = 2
max(a,b)

2

In [33]:
#网络model
class FC_Net(nn.Module):
    def __init__(self,input_dim,n_hidden_1,n_hidden_2,n_hidden_3,output_dim):
        super(FC_Net,self).__init__()
        self.layer1 = nn.Sequential(
                                    
                                    nn.Linear(input_dim,n_hidden_1),
                                    nn.BatchNorm1d(n_hidden_1),
                                    nn.ReLU(True)
                                    )
        self.layer2 = nn.Sequential(
                                    nn.Linear(n_hidden_1,n_hidden_2),
                                    nn.BatchNorm1d(n_hidden_2),
                                    nn.ReLU(True)
                                    )
        self.layer3 = nn.Sequential(
                                    nn.Linear(n_hidden_2,n_hidden_3),
                                    nn.BatchNorm1d(n_hidden_3),
                                    nn.ReLU(True)
                                    )
        self.layer_out = nn.Sequential(
                                    nn.Linear(n_hidden_3,output_dim),
                                    #nn.BatchNorm1d(output_dim),
                                    #nn.ReLU(True)
                                       )

    def forward(self,x):
        #print("x.shape",x.shape)
        x = self.layer1(x)
        #print("x.shape1",x.shape)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer_out(x)
        #x = self.softmax(x)
        #print(x)
        return x

In [63]:
class txt_dataset(data.Dataset):
    def __init__(self,root,transforms0=None):
        self.paths = [root+"\\"+path+"\\"+val for path in os.listdir(root) for val in os.listdir(root+"\\" +path)]
        self.transforms0 = transforms0
        
    def __getitem__(self,index):   
        data_path = self.paths[index]
        label = 1 if 'clear_face' in data_path.split("\\")[-2] else 0
        fr = open(data_path)
        data_str = fr.read()
        array = np.array(data_str.split(",")).astype(float)
        fr.close()
        data = t.from_numpy(array)        
        
        return data,label
    
    def __len__(self):
        return len(self.paths)
                                        

In [15]:
batch_size = 64
learning_rate = 1e-4
num_epoches = 500
weight_decay = 1e-4

In [65]:
dataset0 = txt_dataset("C:\\newfolder43_1_txt6")
dataset0[10000][1]


0

In [9]:
dataloader = DataLoader(dataset0,batch_size=batch_size,shuffle=True)
#dataiter = iter(dataloader)
#imgs,labels = next(dataiter)
#labels

In [16]:
input_dim=4032
n_hidden_1=128
n_hidden_2=64
n_hidden_3=32
output_dim=2
model = FC_Net(input_dim,n_hidden_1,n_hidden_2,n_hidden_3,output_dim)

In [17]:
if t.cuda.is_available():
    model = model.cuda()
    
criterion = nn.CrossEntropyLoss()
criterion.cuda() 
optimizer = optim.Adam(model.parameters(),lr=learning_rate,weight_decay=weight_decay) 

In [15]:
print("training start time  <===>  ",time.localtime())
sum_loss = t.Tensor([0])
for epoch in range(0,num_epoches):
    for i,data in enumerate(dataloader,0):
        data_batch, label = data        
        data_batch, label = Variable(data_batch.float()).cuda(),Variable(label).cuda()
        output = model(data_batch)
        optimizer.zero_grad()
        loss = criterion(output,label)
        loss.backward() 
        optimizer.step()
        
        sum_loss = t.cat((sum_loss,loss.cpu().data),0)
        x = i+epoch*len(dataloader)
        if x%50 == 49:
            loss_aver = sum_loss.mean()
            sum_loss = t.Tensor([0])
            vis.line(X=t.Tensor([x]), Y=t.Tensor([loss_aver]), win='blur_new6',opts={'title': 'loss-batch_new6', 'xlabel':'batch_num','ylabel':'loss_aver'},update='append' if x>49 else None)
            #vis.updateTrace(X=t.Tensor([x]),Y=t.Tensor([loss_aver]),win='face_angle', name='7')
t.save(model.state_dict(),"C:\\Users\\SiChen\\face_sys\\grad_var_full_layers\\model_save\\FC_500_128_64_NoSoftmax_4032_NEWguiyihua_txt_lr1e-4.pth") 
print("training start time  <===>  ",time.localtime())

training start time  <===>   time.struct_time(tm_year=2018, tm_mon=6, tm_mday=27, tm_hour=22, tm_min=38, tm_sec=1, tm_wday=2, tm_yday=178, tm_isdst=0)
training start time  <===>   time.struct_time(tm_year=2018, tm_mon=6, tm_mday=28, tm_hour=4, tm_min=9, tm_sec=48, tm_wday=3, tm_yday=179, tm_isdst=0)


In [37]:
input_dim=8064
n_hidden_1=128
n_hidden_2=64
n_hidden_3=32
output_dim=2
model = FC_Net(input_dim,n_hidden_1,n_hidden_2,n_hidden_3,output_dim)
model.load_state_dict(t.load("C:\\Users\\SiChen\\face_sys\\grad_var_full_layers\\model_save\\FC_500_128_64_NoSoftmax_8064_NEWguiyihua_txt_lr1e-4.pth"))

In [38]:
from torch.utils import data

In [42]:
#带0.5的normalize
img_2grad = np.zeros((28,20,20,3))
var_array = np.zeros((84))
var_array1 = np.zeros((28))
aver_array = np.zeros((84))
aver_array1 = np.zeros((28))
var_array_max_all = 0
var_array_min_all = 100000
aver_array_max_all = 0
aver_array_min_all = 100000
var_array_max1_all = 0
var_array_min1_all = 100000
aver_array_max1_all = 0
aver_array_min1_all = 100000
dir_path = "D:\\newfolder43_2"
save_path = "D:\\newfolder43_3_txt"
dirs = os.listdir(dir_path)
for i in range(len(dirs)):
    pic_names = os.listdir(dir_path+"\\"+dirs[i])
    for n in range(len(pic_names)):
        pic_path = dir_path+"\\"+dirs[i]+"\\"+pic_names[n]
        print(pic_path)
        #pic_path_gbk = pic_path.encode("gbk")
        #pic_path_ascii = pic_path_gbk.decode()
        #image = cv2.imread(pic_path_ascii)
        image=cv2.imdecode(np.fromfile(pic_path,dtype=np.uint8),-1)
        all_result_array = np.array([])
        all_var_array = np.array([])
        all_aver_array = np.array([])
        all_var_array1 = np.array([])
        all_aver_array1 = np.array([])
        for hor_num in range(6):
            for col_num in range(6):
                image1 = image[(20*hor_num):(20*(hor_num+1)),(20*col_num):(20*(col_num+1))]
                image_gaussion = cv2.filter2D(image1,cv2.CV_16S,kernel_)
                for m in range(28):
                    img_2grad[m] = cv2.filter2D(image_gaussion,cv2.CV_16S,p[m])
                    var_array[3*m] = img_2grad[m][:,:,0].std()
                    var_array[3*m+1] = img_2grad[m][:,:,1].std()
                    var_array[3*m+2] = img_2grad[m][:,:,2].std()
                    aver_array[3*m] = img_2grad[m][:,:,0].mean()
                    aver_array[3*m+1] = img_2grad[m][:,:,1].mean()
                    aver_array[3*m+2] = img_2grad[m][:,:,2].mean()
                    var_array1[m] = img_2grad[m].std()
                    aver_array1[m] = img_2grad[m].mean()
                
                all_var_array = np.hstack((all_var_array,var_array))
                all_aver_array = np.hstack((all_aver_array,aver_array))   
                all_var_array1 = np.hstack((all_var_array1,var_array1))
                all_aver_array1 = np.hstack((all_aver_array1,aver_array1))
                        
        all_var_array = np.true_divide((all_var_array-var_array_min_all),(var_array_max_all-var_array_min_all))
        all_aver_array = np.true_divide((all_aver_array-aver_array_min_all),(aver_array_max_all-aver_array_min_all))
        all_var_array1 = np.true_divide((all_var_array1-var_array_min1_all),(var_array_max1_all-var_array_min1_all))
        all_aver_array1 = np.true_divide((all_aver_array1-aver_array_min1_all),(aver_array_max1_all-aver_array_min1_all))
        
        #all_var_array = np.true_divide((all_var_array-0.5),0.5) 
        #all_aver_array = np.true_divide((all_aver_array-0.5),0.5)
        #all_var_array1 = np.true_divide((all_var_array1-0.5),0.5) 
        #all_aver_array1 = np.true_divide((all_aver_array1-0.5),0.5)
        result_array = np.hstack((all_var_array,all_aver_array,all_var_array1,all_aver_array1))
        name = save_path+"\\"+dirs[i]+"\\"+(pic_names[n].split("."))[0]+".txt"
        fw = open(name,"w")
        fw.write(','.join(str(num) for num in result_array))
        fw.close()

D:\newfolder43_2\clear_face\picture_1000_.png
D:\newfolder43_2\clear_face\picture_1002_.png
D:\newfolder43_2\clear_face\picture_1004_.png
D:\newfolder43_2\clear_face\picture_1005_.png
D:\newfolder43_2\clear_face\picture_1008_.png
D:\newfolder43_2\clear_face\picture_1009_.png
D:\newfolder43_2\clear_face\picture_100_.png
D:\newfolder43_2\clear_face\picture_1010_.png
D:\newfolder43_2\clear_face\picture_1013_.png
D:\newfolder43_2\clear_face\picture_1014_.png
D:\newfolder43_2\clear_face\picture_1015_.png
D:\newfolder43_2\clear_face\picture_1016_.png
D:\newfolder43_2\clear_face\picture_1018_.png
D:\newfolder43_2\clear_face\picture_1019_.png
D:\newfolder43_2\clear_face\picture_101_.png
D:\newfolder43_2\clear_face\picture_1020_.png
D:\newfolder43_2\clear_face\picture_1021_.png
D:\newfolder43_2\clear_face\picture_1024_.png
D:\newfolder43_2\clear_face\picture_1025_.png
D:\newfolder43_2\clear_face\picture_1027_.png
D:\newfolder43_2\clear_face\picture_1028_.png
D:\newfolder43_2\clear_face\picture_

D:\newfolder43_2\clear_face\picture_1322_.png
D:\newfolder43_2\clear_face\picture_1326_.png
D:\newfolder43_2\clear_face\picture_1327_.png
D:\newfolder43_2\clear_face\picture_1328_.png
D:\newfolder43_2\clear_face\picture_1329_.png
D:\newfolder43_2\clear_face\picture_132_.png
D:\newfolder43_2\clear_face\picture_1330_.png
D:\newfolder43_2\clear_face\picture_1331_.png
D:\newfolder43_2\clear_face\picture_1332_.png
D:\newfolder43_2\clear_face\picture_1333_.png
D:\newfolder43_2\clear_face\picture_1334_.png
D:\newfolder43_2\clear_face\picture_1335_.png
D:\newfolder43_2\clear_face\picture_1337_.png
D:\newfolder43_2\clear_face\picture_1338_.png
D:\newfolder43_2\clear_face\picture_1339_.png
D:\newfolder43_2\clear_face\picture_1340_.png
D:\newfolder43_2\clear_face\picture_1341_.png
D:\newfolder43_2\clear_face\picture_1342_.png
D:\newfolder43_2\clear_face\picture_1343_.png
D:\newfolder43_2\clear_face\picture_1344_.png
D:\newfolder43_2\clear_face\picture_1345_.png
D:\newfolder43_2\clear_face\picture

D:\newfolder43_2\clear_face\picture_1659_.png
D:\newfolder43_2\clear_face\picture_165_.png
D:\newfolder43_2\clear_face\picture_1661_.png
D:\newfolder43_2\clear_face\picture_1663_.png
D:\newfolder43_2\clear_face\picture_1664_.png
D:\newfolder43_2\clear_face\picture_1666_.png
D:\newfolder43_2\clear_face\picture_1668_.png
D:\newfolder43_2\clear_face\picture_166_.png
D:\newfolder43_2\clear_face\picture_1671_.png
D:\newfolder43_2\clear_face\picture_1673_.png
D:\newfolder43_2\clear_face\picture_1676_.png
D:\newfolder43_2\clear_face\picture_167_.png
D:\newfolder43_2\clear_face\picture_1683_.png
D:\newfolder43_2\clear_face\picture_1684_.png
D:\newfolder43_2\clear_face\picture_1685_.png
D:\newfolder43_2\clear_face\picture_1686_.png
D:\newfolder43_2\clear_face\picture_1688_.png
D:\newfolder43_2\clear_face\picture_168_.png
D:\newfolder43_2\clear_face\picture_1691_.png
D:\newfolder43_2\clear_face\picture_169_.png
D:\newfolder43_2\clear_face\picture_16_.png
D:\newfolder43_2\clear_face\picture_170_.

D:\newfolder43_2\clear_face\picture_2009_.png
D:\newfolder43_2\clear_face\picture_2014_.png
D:\newfolder43_2\clear_face\picture_201_.png
D:\newfolder43_2\clear_face\picture_2021_.png
D:\newfolder43_2\clear_face\picture_2028_.png
D:\newfolder43_2\clear_face\picture_202_.png
D:\newfolder43_2\clear_face\picture_203_.png
D:\newfolder43_2\clear_face\picture_2046_.png
D:\newfolder43_2\clear_face\picture_2051_.png
D:\newfolder43_2\clear_face\picture_2053_.png
D:\newfolder43_2\clear_face\picture_2059_.png
D:\newfolder43_2\clear_face\picture_2061_.png
D:\newfolder43_2\clear_face\picture_2062_.png
D:\newfolder43_2\clear_face\picture_2063_.png
D:\newfolder43_2\clear_face\picture_2064_.png
D:\newfolder43_2\clear_face\picture_2065_.png
D:\newfolder43_2\clear_face\picture_2066_.png
D:\newfolder43_2\clear_face\picture_2068_.png
D:\newfolder43_2\clear_face\picture_2077_.png
D:\newfolder43_2\clear_face\picture_2085_.png
D:\newfolder43_2\clear_face\picture_2088_.png
D:\newfolder43_2\clear_face\picture_2

D:\newfolder43_2\clear_face\picture_2519_.png
D:\newfolder43_2\clear_face\picture_2521_.png
D:\newfolder43_2\clear_face\picture_2525_.png
D:\newfolder43_2\clear_face\picture_2527_.png
D:\newfolder43_2\clear_face\picture_2529_.png
D:\newfolder43_2\clear_face\picture_252_.png
D:\newfolder43_2\clear_face\picture_2531_.png
D:\newfolder43_2\clear_face\picture_2533_.png
D:\newfolder43_2\clear_face\picture_2535_.png
D:\newfolder43_2\clear_face\picture_2537_.png
D:\newfolder43_2\clear_face\picture_2541_.png
D:\newfolder43_2\clear_face\picture_2547_.png
D:\newfolder43_2\clear_face\picture_2549_.png
D:\newfolder43_2\clear_face\picture_2550_.png
D:\newfolder43_2\clear_face\picture_2552_.png
D:\newfolder43_2\clear_face\picture_2559_.png
D:\newfolder43_2\clear_face\picture_2565_.png
D:\newfolder43_2\clear_face\picture_2578_.png
D:\newfolder43_2\clear_face\picture_2586_.png
D:\newfolder43_2\clear_face\picture_2587_.png
D:\newfolder43_2\clear_face\picture_2593_.png
D:\newfolder43_2\clear_face\picture

...

D:\newfolder43_2\fuzzy_face\picture_2208_.png
D:\newfolder43_2\fuzzy_face\picture_220_.png
D:\newfolder43_2\fuzzy_face\picture_2210_.png
D:\newfolder43_2\fuzzy_face\picture_2211_.png
D:\newfolder43_2\fuzzy_face\picture_2215_.png
D:\newfolder43_2\fuzzy_face\picture_221_.png
D:\newfolder43_2\fuzzy_face\picture_2223_.png
D:\newfolder43_2\fuzzy_face\picture_2224_.png
D:\newfolder43_2\fuzzy_face\picture_2228_.png
D:\newfolder43_2\fuzzy_face\picture_222_.png
D:\newfolder43_2\fuzzy_face\picture_2230_.png
D:\newfolder43_2\fuzzy_face\picture_2236_.png
D:\newfolder43_2\fuzzy_face\picture_2238_.png
D:\newfolder43_2\fuzzy_face\picture_223_.png
D:\newfolder43_2\fuzzy_face\picture_2242_.png
D:\newfolder43_2\fuzzy_face\picture_2246_.png
D:\newfolder43_2\fuzzy_face\picture_2248_.png
D:\newfolder43_2\fuzzy_face\picture_224_.png
D:\newfolder43_2\fuzzy_face\picture_2252_.png
D:\newfolder43_2\fuzzy_face\picture_2256_.png
D:\newfolder43_2\fuzzy_face\picture_225_.png
D:\newfolder43_2\fuzzy_face\picture_2262

D:\newfolder43_2\fuzzy_face\picture_2525_.png
D:\newfolder43_2\fuzzy_face\picture_2526_.png
D:\newfolder43_2\fuzzy_face\picture_2527_.png
D:\newfolder43_2\fuzzy_face\picture_2528_.png
D:\newfolder43_2\fuzzy_face\picture_2529_.png
D:\newfolder43_2\fuzzy_face\picture_252_.png
D:\newfolder43_2\fuzzy_face\picture_2530_.png
D:\newfolder43_2\fuzzy_face\picture_2531_.png
D:\newfolder43_2\fuzzy_face\picture_2532_.png
D:\newfolder43_2\fuzzy_face\picture_2533_.png
D:\newfolder43_2\fuzzy_face\picture_2534_.png
D:\newfolder43_2\fuzzy_face\picture_2535_.png
D:\newfolder43_2\fuzzy_face\picture_2536_.png
D:\newfolder43_2\fuzzy_face\picture_2537_.png
D:\newfolder43_2\fuzzy_face\picture_2538_.png
D:\newfolder43_2\fuzzy_face\picture_2539_.png
D:\newfolder43_2\fuzzy_face\picture_253_.png
D:\newfolder43_2\fuzzy_face\picture_2540_.png
D:\newfolder43_2\fuzzy_face\picture_2541_.png
D:\newfolder43_2\fuzzy_face\picture_2542_.png
D:\newfolder43_2\fuzzy_face\picture_2543_.png
D:\newfolder43_2\fuzzy_face\picture_

D:\newfolder43_2\fuzzy_face\picture_3014_.png
D:\newfolder43_2\fuzzy_face\picture_301_.png
D:\newfolder43_2\fuzzy_face\picture_302_.png
D:\newfolder43_2\fuzzy_face\picture_303_.png
D:\newfolder43_2\fuzzy_face\picture_3047_.png
D:\newfolder43_2\fuzzy_face\picture_304_.png
D:\newfolder43_2\fuzzy_face\picture_305_.png
D:\newfolder43_2\fuzzy_face\picture_306_.png
D:\newfolder43_2\fuzzy_face\picture_307_.png
D:\newfolder43_2\fuzzy_face\picture_308_.png
D:\newfolder43_2\fuzzy_face\picture_3091_.png
D:\newfolder43_2\fuzzy_face\picture_3093_.png
D:\newfolder43_2\fuzzy_face\picture_3095_.png
D:\newfolder43_2\fuzzy_face\picture_3099_.png
D:\newfolder43_2\fuzzy_face\picture_309_.png
D:\newfolder43_2\fuzzy_face\picture_30_.png
D:\newfolder43_2\fuzzy_face\picture_3101_.png
D:\newfolder43_2\fuzzy_face\picture_3103_.png
D:\newfolder43_2\fuzzy_face\picture_3107_.png
D:\newfolder43_2\fuzzy_face\picture_310_.png
D:\newfolder43_2\fuzzy_face\picture_311_.png
D:\newfolder43_2\fuzzy_face\picture_3128_.png
D

D:\newfolder43_2\fuzzy_face\picture_424_.png
D:\newfolder43_2\fuzzy_face\picture_425_.png
D:\newfolder43_2\fuzzy_face\picture_426_.png
D:\newfolder43_2\fuzzy_face\picture_427_.png
D:\newfolder43_2\fuzzy_face\picture_42_.png
D:\newfolder43_2\fuzzy_face\picture_438_.png
D:\newfolder43_2\fuzzy_face\picture_43_.png
D:\newfolder43_2\fuzzy_face\picture_443_.png
D:\newfolder43_2\fuzzy_face\picture_444_.png
D:\newfolder43_2\fuzzy_face\picture_44_.png
D:\newfolder43_2\fuzzy_face\picture_453_.png
D:\newfolder43_2\fuzzy_face\picture_454_.png
D:\newfolder43_2\fuzzy_face\picture_45_.png
D:\newfolder43_2\fuzzy_face\picture_46_.png
D:\newfolder43_2\fuzzy_face\picture_471_.png
D:\newfolder43_2\fuzzy_face\picture_472_.png
D:\newfolder43_2\fuzzy_face\picture_486_.png
D:\newfolder43_2\fuzzy_face\picture_488_.png
D:\newfolder43_2\fuzzy_face\picture_495_.png
D:\newfolder43_2\fuzzy_face\picture_497_.png
D:\newfolder43_2\fuzzy_face\picture_498_.png
D:\newfolder43_2\fuzzy_face\picture_49_.png
D:\newfolder43_2

D:\newfolder43_2\fuzzy_face\picture_752_.png
D:\newfolder43_2\fuzzy_face\picture_753_.png
D:\newfolder43_2\fuzzy_face\picture_754_.png
D:\newfolder43_2\fuzzy_face\picture_755_.png
D:\newfolder43_2\fuzzy_face\picture_756_.png
D:\newfolder43_2\fuzzy_face\picture_757_.png
D:\newfolder43_2\fuzzy_face\picture_758_.png
D:\newfolder43_2\fuzzy_face\picture_759_.png
D:\newfolder43_2\fuzzy_face\picture_75_.png
D:\newfolder43_2\fuzzy_face\picture_760_.png
D:\newfolder43_2\fuzzy_face\picture_761_.png
D:\newfolder43_2\fuzzy_face\picture_762_.png
D:\newfolder43_2\fuzzy_face\picture_763_.png
D:\newfolder43_2\fuzzy_face\picture_764_.png
D:\newfolder43_2\fuzzy_face\picture_765_.png
D:\newfolder43_2\fuzzy_face\picture_766_.png
D:\newfolder43_2\fuzzy_face\picture_767_.png
D:\newfolder43_2\fuzzy_face\picture_768_.png
D:\newfolder43_2\fuzzy_face\picture_769_.png
D:\newfolder43_2\fuzzy_face\picture_770_.png
D:\newfolder43_2\fuzzy_face\picture_771_.png
D:\newfolder43_2\fuzzy_face\picture_772_.png
D:\newfolde

In [28]:
#带0.5的normalize
img_2grad = np.zeros((28,30,30,3))
var_array = np.zeros((84))
var_array1 = np.zeros((28))
aver_array = np.zeros((84))
aver_array1 = np.zeros((28))
var_array_max_all = 0
var_array_min_all = 100000
aver_array_max_all = 0
aver_array_min_all = 100000
var_array_max1_all = 0
var_array_min1_all = 100000
aver_array_max1_all = 0
aver_array_min1_all = 100000
dir_path = "D:\\newfolder43_2"
save_path = "D:\\newfolder43_3_txt"
dirs = os.listdir(dir_path)
for i in range(len(dirs)):
    pic_names = os.listdir(dir_path+"\\"+dirs[i])
    for n in range(len(pic_names)):
        pic_path = dir_path+"\\"+dirs[i]+"\\"+pic_names[n]
        print(pic_path)
        #pic_path_gbk = pic_path.encode("gbk")
        #pic_path_ascii = pic_path_gbk.decode()
        #image = cv2.imread(pic_path_ascii)
        image=cv2.imdecode(np.fromfile(pic_path,dtype=np.uint8),-1)
        all_result_array = np.array([])
        all_var_array = np.array([])
        all_aver_array = np.array([])
        all_var_array1 = np.array([])
        all_aver_array1 = np.array([])
        for hor_num in range(4):
            for col_num in range(4):
                image1 = image[(30*hor_num):(30*(hor_num+1)),(30*col_num):(30*(col_num+1))]
                image_gaussion = cv2.filter2D(image1,cv2.CV_16S,kernel_)
                for m in range(28):
                    img_2grad[m] = cv2.filter2D(image_gaussion,cv2.CV_16S,p[m])
                    var_array[3*m] = img_2grad[m][:,:,0].std()
                    var_array[3*m+1] = img_2grad[m][:,:,1].std()
                    var_array[3*m+2] = img_2grad[m][:,:,2].std()
                    #aver_array[3*m] = img_2grad[m][:,:,0].mean()
                    #aver_array[3*m+1] = img_2grad[m][:,:,1].mean()
                    #aver_array[3*m+2] = img_2grad[m][:,:,2].mean()
                    var_array1[m] = img_2grad[m].std()
                    #aver_array1[m] = img_2grad[m].mean()
                
                all_var_array = np.hstack((all_var_array,var_array))
                #all_aver_array = np.hstack((all_aver_array,aver_array))   
                all_var_array1 = np.hstack((all_var_array1,var_array1))
                #all_aver_array1 = np.hstack((all_aver_array1,aver_array1))
                        
        all_var_array = np.true_divide((all_var_array-var_array_min_all),(var_array_max_all-var_array_min_all))
        all_aver_array = np.true_divide((all_aver_array-aver_array_min_all),(aver_array_max_all-aver_array_min_all))
        all_var_array1 = np.true_divide((all_var_array1-var_array_min1_all),(var_array_max1_all-var_array_min1_all))
        all_aver_array1 = np.true_divide((all_aver_array1-aver_array_min1_all),(aver_array_max1_all-aver_array_min1_all))
        
        #all_var_array = np.true_divide((all_var_array-0.5),0.5) 
        #all_aver_array = np.true_divide((all_aver_array-0.5),0.5)
        #all_var_array1 = np.true_divide((all_var_array1-0.5),0.5) 
        #all_aver_array1 = np.true_divide((all_aver_array1-0.5),0.5)
        result_array = np.hstack((all_var_array,all_aver_array,all_var_array1,all_aver_array1))
        name = save_path+"\\"+dirs[i]+"\\"+(pic_names[n].split("."))[0]+".txt"
        fw = open(name,"w")
        fw.write(','.join(str(num) for num in result_array))
        fw.close()

D:\newfolder43_2\clear_face\picture_1000_.png
D:\newfolder43_2\clear_face\picture_1002_.png
D:\newfolder43_2\clear_face\picture_1004_.png
D:\newfolder43_2\clear_face\picture_1005_.png
D:\newfolder43_2\clear_face\picture_1008_.png
D:\newfolder43_2\clear_face\picture_1009_.png
D:\newfolder43_2\clear_face\picture_100_.png
D:\newfolder43_2\clear_face\picture_1010_.png
D:\newfolder43_2\clear_face\picture_1013_.png
D:\newfolder43_2\clear_face\picture_1014_.png
D:\newfolder43_2\clear_face\picture_1015_.png
D:\newfolder43_2\clear_face\picture_1016_.png
D:\newfolder43_2\clear_face\picture_1018_.png
D:\newfolder43_2\clear_face\picture_1019_.png
D:\newfolder43_2\clear_face\picture_101_.png
D:\newfolder43_2\clear_face\picture_1020_.png
D:\newfolder43_2\clear_face\picture_1021_.png
D:\newfolder43_2\clear_face\picture_1024_.png
D:\newfolder43_2\clear_face\picture_1025_.png
D:\newfolder43_2\clear_face\picture_1027_.png
D:\newfolder43_2\clear_face\picture_1028_.png
D:\newfolder43_2\clear_face\picture_

D:\newfolder43_2\clear_face\picture_1326_.png
D:\newfolder43_2\clear_face\picture_1327_.png
D:\newfolder43_2\clear_face\picture_1328_.png
D:\newfolder43_2\clear_face\picture_1329_.png
D:\newfolder43_2\clear_face\picture_132_.png
D:\newfolder43_2\clear_face\picture_1330_.png
D:\newfolder43_2\clear_face\picture_1331_.png
D:\newfolder43_2\clear_face\picture_1332_.png
D:\newfolder43_2\clear_face\picture_1333_.png
D:\newfolder43_2\clear_face\picture_1334_.png
D:\newfolder43_2\clear_face\picture_1335_.png
D:\newfolder43_2\clear_face\picture_1337_.png
D:\newfolder43_2\clear_face\picture_1338_.png
D:\newfolder43_2\clear_face\picture_1339_.png
D:\newfolder43_2\clear_face\picture_1340_.png
D:\newfolder43_2\clear_face\picture_1341_.png
D:\newfolder43_2\clear_face\picture_1342_.png
D:\newfolder43_2\clear_face\picture_1343_.png
D:\newfolder43_2\clear_face\picture_1344_.png
D:\newfolder43_2\clear_face\picture_1345_.png
D:\newfolder43_2\clear_face\picture_1346_.png
D:\newfolder43_2\clear_face\picture

D:\newfolder43_2\clear_face\picture_1661_.png
D:\newfolder43_2\clear_face\picture_1663_.png
D:\newfolder43_2\clear_face\picture_1664_.png
D:\newfolder43_2\clear_face\picture_1666_.png
D:\newfolder43_2\clear_face\picture_1668_.png
D:\newfolder43_2\clear_face\picture_166_.png
D:\newfolder43_2\clear_face\picture_1671_.png
D:\newfolder43_2\clear_face\picture_1673_.png
D:\newfolder43_2\clear_face\picture_1676_.png
D:\newfolder43_2\clear_face\picture_167_.png
D:\newfolder43_2\clear_face\picture_1683_.png
D:\newfolder43_2\clear_face\picture_1684_.png
D:\newfolder43_2\clear_face\picture_1685_.png
D:\newfolder43_2\clear_face\picture_1686_.png
D:\newfolder43_2\clear_face\picture_1688_.png
D:\newfolder43_2\clear_face\picture_168_.png
D:\newfolder43_2\clear_face\picture_1691_.png
D:\newfolder43_2\clear_face\picture_169_.png
D:\newfolder43_2\clear_face\picture_16_.png
D:\newfolder43_2\clear_face\picture_170_.png
D:\newfolder43_2\clear_face\picture_171_.png
D:\newfolder43_2\clear_face\picture_173_.p

D:\newfolder43_2\clear_face\picture_2021_.png
D:\newfolder43_2\clear_face\picture_2028_.png
D:\newfolder43_2\clear_face\picture_202_.png
D:\newfolder43_2\clear_face\picture_203_.png
D:\newfolder43_2\clear_face\picture_2046_.png
D:\newfolder43_2\clear_face\picture_2051_.png
D:\newfolder43_2\clear_face\picture_2053_.png
D:\newfolder43_2\clear_face\picture_2059_.png
D:\newfolder43_2\clear_face\picture_2061_.png
D:\newfolder43_2\clear_face\picture_2062_.png
D:\newfolder43_2\clear_face\picture_2063_.png
D:\newfolder43_2\clear_face\picture_2064_.png
D:\newfolder43_2\clear_face\picture_2065_.png
D:\newfolder43_2\clear_face\picture_2066_.png
D:\newfolder43_2\clear_face\picture_2068_.png
D:\newfolder43_2\clear_face\picture_2077_.png
D:\newfolder43_2\clear_face\picture_2085_.png
D:\newfolder43_2\clear_face\picture_2088_.png
D:\newfolder43_2\clear_face\picture_209_.png
D:\newfolder43_2\clear_face\picture_20_.png
D:\newfolder43_2\clear_face\picture_2107_.png
D:\newfolder43_2\clear_face\picture_211

D:\newfolder43_2\clear_face\picture_2529_.png
D:\newfolder43_2\clear_face\picture_252_.png
D:\newfolder43_2\clear_face\picture_2531_.png
D:\newfolder43_2\clear_face\picture_2533_.png
D:\newfolder43_2\clear_face\picture_2535_.png
D:\newfolder43_2\clear_face\picture_2537_.png
D:\newfolder43_2\clear_face\picture_2541_.png
D:\newfolder43_2\clear_face\picture_2547_.png
D:\newfolder43_2\clear_face\picture_2549_.png
D:\newfolder43_2\clear_face\picture_2550_.png
D:\newfolder43_2\clear_face\picture_2552_.png
D:\newfolder43_2\clear_face\picture_2559_.png
D:\newfolder43_2\clear_face\picture_2565_.png
D:\newfolder43_2\clear_face\picture_2578_.png
D:\newfolder43_2\clear_face\picture_2586_.png
D:\newfolder43_2\clear_face\picture_2587_.png
D:\newfolder43_2\clear_face\picture_2593_.png
D:\newfolder43_2\clear_face\picture_2595_.png
D:\newfolder43_2\clear_face\picture_2599_.png
D:\newfolder43_2\clear_face\picture_25_.png
D:\newfolder43_2\clear_face\picture_2602_.png
D:\newfolder43_2\clear_face\picture_2

...

D:\newfolder43_2\fuzzy_face\picture_2236_.png
D:\newfolder43_2\fuzzy_face\picture_2238_.png
D:\newfolder43_2\fuzzy_face\picture_223_.png
D:\newfolder43_2\fuzzy_face\picture_2242_.png
D:\newfolder43_2\fuzzy_face\picture_2246_.png
D:\newfolder43_2\fuzzy_face\picture_2248_.png
D:\newfolder43_2\fuzzy_face\picture_224_.png
D:\newfolder43_2\fuzzy_face\picture_2252_.png
D:\newfolder43_2\fuzzy_face\picture_2256_.png
D:\newfolder43_2\fuzzy_face\picture_225_.png
D:\newfolder43_2\fuzzy_face\picture_2262_.png
D:\newfolder43_2\fuzzy_face\picture_2268_.png
D:\newfolder43_2\fuzzy_face\picture_226_.png
D:\newfolder43_2\fuzzy_face\picture_2273_.png
D:\newfolder43_2\fuzzy_face\picture_2274_.png
D:\newfolder43_2\fuzzy_face\picture_2279_.png
D:\newfolder43_2\fuzzy_face\picture_227_.png
D:\newfolder43_2\fuzzy_face\picture_2285_.png
D:\newfolder43_2\fuzzy_face\picture_2288_.png
D:\newfolder43_2\fuzzy_face\picture_228_.png
D:\newfolder43_2\fuzzy_face\picture_2291_.png
D:\newfolder43_2\fuzzy_face\picture_2295

D:\newfolder43_2\fuzzy_face\picture_2536_.png
D:\newfolder43_2\fuzzy_face\picture_2537_.png
D:\newfolder43_2\fuzzy_face\picture_2538_.png
D:\newfolder43_2\fuzzy_face\picture_2539_.png
D:\newfolder43_2\fuzzy_face\picture_253_.png
D:\newfolder43_2\fuzzy_face\picture_2540_.png
D:\newfolder43_2\fuzzy_face\picture_2541_.png
D:\newfolder43_2\fuzzy_face\picture_2542_.png
D:\newfolder43_2\fuzzy_face\picture_2543_.png
D:\newfolder43_2\fuzzy_face\picture_2544_.png
D:\newfolder43_2\fuzzy_face\picture_2545_.png
D:\newfolder43_2\fuzzy_face\picture_2546_.png
D:\newfolder43_2\fuzzy_face\picture_2547_.png
D:\newfolder43_2\fuzzy_face\picture_2548_.png
D:\newfolder43_2\fuzzy_face\picture_2549_.png
D:\newfolder43_2\fuzzy_face\picture_254_.png
D:\newfolder43_2\fuzzy_face\picture_2550_.png
D:\newfolder43_2\fuzzy_face\picture_2551_.png
D:\newfolder43_2\fuzzy_face\picture_2552_.png
D:\newfolder43_2\fuzzy_face\picture_2553_.png
D:\newfolder43_2\fuzzy_face\picture_2554_.png
D:\newfolder43_2\fuzzy_face\picture_

D:\newfolder43_2\fuzzy_face\picture_3095_.png
D:\newfolder43_2\fuzzy_face\picture_3099_.png
D:\newfolder43_2\fuzzy_face\picture_309_.png
D:\newfolder43_2\fuzzy_face\picture_30_.png
D:\newfolder43_2\fuzzy_face\picture_3101_.png
D:\newfolder43_2\fuzzy_face\picture_3103_.png
D:\newfolder43_2\fuzzy_face\picture_3107_.png
D:\newfolder43_2\fuzzy_face\picture_310_.png
D:\newfolder43_2\fuzzy_face\picture_311_.png
D:\newfolder43_2\fuzzy_face\picture_3128_.png
D:\newfolder43_2\fuzzy_face\picture_312_.png
D:\newfolder43_2\fuzzy_face\picture_313_.png
D:\newfolder43_2\fuzzy_face\picture_314_.png
D:\newfolder43_2\fuzzy_face\picture_315_.png
D:\newfolder43_2\fuzzy_face\picture_316_.png
D:\newfolder43_2\fuzzy_face\picture_317_.png
D:\newfolder43_2\fuzzy_face\picture_318_.png
D:\newfolder43_2\fuzzy_face\picture_31_.png
D:\newfolder43_2\fuzzy_face\picture_3202_.png
D:\newfolder43_2\fuzzy_face\picture_3203_.png
D:\newfolder43_2\fuzzy_face\picture_3204_.png
D:\newfolder43_2\fuzzy_face\picture_3205_.png
D:

D:\newfolder43_2\fuzzy_face\picture_45_.png
D:\newfolder43_2\fuzzy_face\picture_46_.png
D:\newfolder43_2\fuzzy_face\picture_471_.png
D:\newfolder43_2\fuzzy_face\picture_472_.png
D:\newfolder43_2\fuzzy_face\picture_486_.png
D:\newfolder43_2\fuzzy_face\picture_488_.png
D:\newfolder43_2\fuzzy_face\picture_495_.png
D:\newfolder43_2\fuzzy_face\picture_497_.png
D:\newfolder43_2\fuzzy_face\picture_498_.png
D:\newfolder43_2\fuzzy_face\picture_49_.png
D:\newfolder43_2\fuzzy_face\picture_502_.png
D:\newfolder43_2\fuzzy_face\picture_503_.png
D:\newfolder43_2\fuzzy_face\picture_504_.png
D:\newfolder43_2\fuzzy_face\picture_505_.png
D:\newfolder43_2\fuzzy_face\picture_506_.png
D:\newfolder43_2\fuzzy_face\picture_50_.png
D:\newfolder43_2\fuzzy_face\picture_515_.png
D:\newfolder43_2\fuzzy_face\picture_51_.png
D:\newfolder43_2\fuzzy_face\picture_520_.png
D:\newfolder43_2\fuzzy_face\picture_521_.png
D:\newfolder43_2\fuzzy_face\picture_523_.png
D:\newfolder43_2\fuzzy_face\picture_524_.png
D:\newfolder43_

D:\newfolder43_2\fuzzy_face\picture_764_.png
D:\newfolder43_2\fuzzy_face\picture_765_.png
D:\newfolder43_2\fuzzy_face\picture_766_.png
D:\newfolder43_2\fuzzy_face\picture_767_.png
D:\newfolder43_2\fuzzy_face\picture_768_.png
D:\newfolder43_2\fuzzy_face\picture_769_.png
D:\newfolder43_2\fuzzy_face\picture_770_.png
D:\newfolder43_2\fuzzy_face\picture_771_.png
D:\newfolder43_2\fuzzy_face\picture_772_.png
D:\newfolder43_2\fuzzy_face\picture_773_.png
D:\newfolder43_2\fuzzy_face\picture_774_.png
D:\newfolder43_2\fuzzy_face\picture_775_.png
D:\newfolder43_2\fuzzy_face\picture_776_.png
D:\newfolder43_2\fuzzy_face\picture_777_.png
D:\newfolder43_2\fuzzy_face\picture_779_.png
D:\newfolder43_2\fuzzy_face\picture_780_.png
D:\newfolder43_2\fuzzy_face\picture_781_.png
D:\newfolder43_2\fuzzy_face\picture_782_.png
D:\newfolder43_2\fuzzy_face\picture_783_.png
D:\newfolder43_2\fuzzy_face\picture_784_.png
D:\newfolder43_2\fuzzy_face\picture_785_.png
D:\newfolder43_2\fuzzy_face\picture_786_.png
D:\newfold

In [69]:
#这段测试代码，仍然可以用在txt为训练集的网络中，因为使用var_dataset()返回的也是224个数据，恰好和从txt文件中读回来的一样。
from PIL import Image
import shutil

model.cuda()
model.eval()

test_right_num = 0
test_right_num = t.Tensor([test_right_num]).int()
test_right_num = Variable(test_right_num)

test_dataset1 = txt_dataset("D:\\newfolder43_3_txt")
i =0
for img0, label0 in test_dataset1:
    img0 = Variable(img0.float()).view(1,8064).cuda()
    label0 = Variable(t.Tensor([label0]).long()).cuda()
    output = model(img0)
    print(output)
    i=i+1
    print(i)
    _,pred = t.min(output.cpu(),1)
    #print(pred)
    num_correct = (pred==label0.cpu()).sum()
    num_correct = num_correct.int()
    print()
    test_right_num = test_right_num + num_correct
    print(test_right_num)
    
acc = (float(test_right_num))/len(test_dataset)
print(test_right_num)
print(len(test_dataset))
print(acc)    
    

Variable containing:
 14602.2549 -14547.2275
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

1

Variable containing:
 1
[torch.IntTensor of size 1]

Variable containing:
 14602.2949 -14547.2656
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

2

Variable containing:
 2
[torch.IntTensor of size 1]

Variable containing:
 14599.4678 -14544.4570
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3

Variable containing:
 3
[torch.IntTensor of size 1]

Variable containing:
 14599.4688 -14544.4609
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

4

Variable containing:
 4
[torch.IntTensor of size 1]

Variable containing:
 14601.8486 -14546.8389
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

5

Variable containing:
 5
[torch.IntTensor of size 1]

Variable containing:
 14601.2871 -14546.2861
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

6

Variable containing:
 6
[torch.IntTensor of size 1]

Variable containing:
 14593.6748 -14538.7334
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

7

Variable containing:
 7


Variable containing:
 14536.8408 -14482.2354
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

60

Variable containing:
 60
[torch.IntTensor of size 1]

Variable containing:
 14600.5537 -14545.5410
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

61

Variable containing:
 61
[torch.IntTensor of size 1]

Variable containing:
 14604.2461 -14549.2148
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

62

Variable containing:
 62
[torch.IntTensor of size 1]

Variable containing:
 14534.6133 -14480.0244
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

63

Variable containing:
 63
[torch.IntTensor of size 1]

Variable containing:
 14570.6484 -14515.8311
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

64

Variable containing:
 64
[torch.IntTensor of size 1]

Variable containing:
 14545.1641 -14490.5293
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

65

Variable containing:
 65
[torch.IntTensor of size 1]

Variable containing:
 14563.2568 -14508.4658
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

66

Variable co

Variable containing:
 14565.3916 -14510.6191
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

124

Variable containing:
 124
[torch.IntTensor of size 1]

Variable containing:
 14589.5801 -14534.6289
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

125

Variable containing:
 125
[torch.IntTensor of size 1]

Variable containing:
 14585.1084 -14530.2012
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

126

Variable containing:
 126
[torch.IntTensor of size 1]

Variable containing:
 14579.0742 -14524.1816
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

127

Variable containing:
 127
[torch.IntTensor of size 1]

Variable containing:
 14580.5898 -14525.7197
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

128

Variable containing:
 128
[torch.IntTensor of size 1]

Variable containing:
 14579.3535 -14524.4580
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

129

Variable containing:
 129
[torch.IntTensor of size 1]

Variable containing:
 14586.5527 -14531.6201
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

130

Variable containing:
 14557.8496 -14503.1416
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

186

Variable containing:
 186
[torch.IntTensor of size 1]

Variable containing:
 14566.6377 -14511.8652
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

187

Variable containing:
 187
[torch.IntTensor of size 1]

Variable containing:
 14557.7090 -14503.0010
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

188

Variable containing:
 188
[torch.IntTensor of size 1]

Variable containing:
 14568.1387 -14513.3613
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

189

Variable containing:
 189
[torch.IntTensor of size 1]

Variable containing:
 14566.4805 -14511.7090
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

190

Variable containing:
 190
[torch.IntTensor of size 1]

Variable containing:
 14596.1143 -14541.1279
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

191

Variable containing:
 191
[torch.IntTensor of size 1]

Variable containing:
 14568.9785 -14514.1787
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

192

Variable containing:
 14571.1797 -14516.3525
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

250

Variable containing:
 250
[torch.IntTensor of size 1]

Variable containing:
 14565.3945 -14510.6074
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

251

Variable containing:
 251
[torch.IntTensor of size 1]

Variable containing:
 14586.8750 -14531.9561
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

252

Variable containing:
 252
[torch.IntTensor of size 1]

Variable containing:
 14537.0400 -14482.4297
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

253

Variable containing:
 253
[torch.IntTensor of size 1]

Variable containing:
 14584.3066 -14529.3867
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

254

Variable containing:
 254
[torch.IntTensor of size 1]

Variable containing:
 14574.6309 -14519.7910
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

255

Variable containing:
 255
[torch.IntTensor of size 1]

Variable containing:
 14582.8672 -14527.9473
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

256

...

Variable containing:
 14613.8115 -14558.7256
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

2896

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14620.3037 -14565.1914
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

2897

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14609.3320 -14554.2734
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

2898

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14619.8730 -14564.7607
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

2899

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14603.2773 -14548.2461
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

2900

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14620.3809 -14565.2217
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

2901

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14602.9102 -14547.8760
[torch.cuda.FloatTensor of size 1x2 (

Variable containing:
 14601.6045 -14546.5879
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

2956

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14606.0713 -14551.0342
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

2957

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14613.2246 -14558.1396
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

2958

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14617.7207 -14562.6035
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

2959

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14611.4473 -14556.3682
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

2960

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14611.7393 -14556.6670
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

2961

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14611.6211 -14556.5547
[torch.cuda.FloatTensor of size 1x2 (

Variable containing:
 14594.0898 -14539.1494
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3015

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14604.4648 -14549.4473
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3016

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14604.5742 -14549.5332
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3017

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14595.6289 -14540.6650
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3018

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14594.9062 -14539.9385
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3019

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14597.8379 -14542.8818
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3020

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14591.5645 -14536.5908
[torch.cuda.FloatTensor of size 1x2 (


Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14607.3379 -14552.2861
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3077

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14600.3594 -14545.3750
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3078

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14607.3604 -14552.3057
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3079

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14612.4512 -14557.3799
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3080

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14587.7715 -14532.8584
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3081

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14611.1719 -14556.1045
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3082

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 146

Variable containing:
 14607.2080 -14552.1963
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3139

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14612.7754 -14557.7070
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3140

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14611.7920 -14556.7275
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3141

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14616.8281 -14561.7158
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3142

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14622.5244 -14567.3838
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3143

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14613.3535 -14558.2734
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

3144

Variable containing:
 1969
[torch.IntTensor of size 1]

Variable containing:
 14620.4404 -14565.3311
[torch.cuda.FloatTensor of size 1x2 (